In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm

from utils.data import *
from utils.metrics import MAPE, WAPE, MAE
from utils.dl import *

In [2]:
dataset_path = "data/huge_stock_market_dataset/"

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

lags = 32
horizon = 8
stride = 1
batch_size = 256
val_size = 0.15
test_size = 0.0
drop_last = False
features = 1
epochs = 200
verbose = False

model_params = {'num_channels': [128] * 4, 'kernel_size': 2, 'dropout': 0.25, 'output_size': horizon, 'input_size': lags}

cuda:0


In [4]:
ts_iterator = get_hsm_dataset(dataset_path, selected_files=f"{dataset_path}/selected.csv")

results = []
for time_series in tqdm(ts_iterator):
    train_dl, val_dl, test_dl, X_scaler, y_scaler = create_ts_dl(time_series[["Close"]], time_series["Close"], lags=lags, horizon=horizon, stride=stride,\
                                            batch_size=batch_size, device=device, data_preprocess="log_returns",\
                                            val_size=val_size, test_size=test_size, drop_last=drop_last)
    
    model = Model(seed=0, device=device)
    model.set_model(TCN, **model_params)
    optim_params = {'params': model.model.parameters(), 'lr': 4e-4}
    model.set_optim(torch.optim.AdamW, **optim_params)
    model.set_criterion(MAE)

    model.train(train_dl, epochs=epochs, print_info=verbose, agg_loss="mean")
    results.append({"train": model.eval(train_dl, agg_loss="mean"), "val": model.eval(val_dl, agg_loss="mean")})

    del model, train_dl, val_dl, test_dl
    torch.cuda.empty_cache()

0it [00:00, ?it/s]Global seed set to 0
1it [00:12, 12.19s/it]Global seed set to 0
2it [00:18,  8.76s/it]Global seed set to 0
3it [00:24,  7.56s/it]Global seed set to 0
4it [00:30,  6.98s/it]Global seed set to 0
5it [00:34,  5.95s/it]Global seed set to 0
6it [00:47,  8.19s/it]Global seed set to 0
7it [00:57,  8.67s/it]Global seed set to 0
8it [01:06,  9.01s/it]Global seed set to 0
9it [01:18,  9.88s/it]Global seed set to 0
10it [01:28,  9.89s/it]Global seed set to 0
11it [01:46, 12.25s/it]Global seed set to 0
12it [02:03, 13.91s/it]Global seed set to 0
13it [02:21, 15.16s/it]Global seed set to 0
14it [02:38, 15.48s/it]Global seed set to 0
15it [02:57, 16.62s/it]Global seed set to 0
16it [03:16, 17.52s/it]Global seed set to 0
17it [03:38, 18.71s/it]Global seed set to 0
18it [04:00, 19.72s/it]Global seed set to 0
19it [04:21, 20.22s/it]Global seed set to 0
20it [04:43, 20.56s/it]Global seed set to 0
21it [05:57, 36.70s/it]Global seed set to 0
22it [07:14, 48.84s/it]Global seed set to 0
23

In [5]:
results

[{'train': 0.07601827010512352, 'val': 0.09294126182794571},
 {'train': 0.03591239328185717, 'val': 0.05265411362051964},
 {'train': 0.006942708821346362, 'val': 0.005144833587110043},
 {'train': 0.009543801036973795, 'val': 0.010662950575351715},
 {'train': 0.01589103927835822, 'val': 0.013669661246240139},
 {'train': 0.026791866247852642, 'val': 0.027336619794368744},
 {'train': 0.02450627014040947, 'val': 0.036064375191926956},
 {'train': 0.0166728213429451, 'val': 0.015392552129924297},
 {'train': 0.008599746196220318, 'val': 0.0061883225571364164},
 {'train': 0.007930880784988404, 'val': 0.0070607042871415615},
 {'train': 0.047204966760343976, 'val': 0.039447009563446045},
 {'train': 0.034146616235375404, 'val': 0.021188664250075817},
 {'train': 0.03233239323728614, 'val': 0.032122411765158176},
 {'train': 0.011525885667651892, 'val': 0.006635635392740369},
 {'train': 0.017489527072757482, 'val': 0.01032670121639967},
 {'train': 0.03790958225727081, 'val': 0.027314666658639908},
 

In [7]:
pd.DataFrame(results).to_csv("results\\pure_TCN.csv", index=False)

# Augmentation with QuantGAN synthetic data

In [4]:
synthetic_path = f"{dataset_path}synthetic/QuantGAN/"

In [4]:
class CombinedDataLoader:
    def __init__(self, *dls):
        self.dls = dls

    def __len__(self):
        return sum(map(len, self.dls))
    
    def __iter__(self):
        for dl in self.dls:
            for v in dl:
                yield v

In [6]:
ts_iterator = get_hsm_dataset(dataset_path, selected_files=f"{dataset_path}/selected.csv")
epochs = 2

results = []
for ts_index, time_series in tqdm(enumerate(ts_iterator)):
    train_dl, val_dl, test_dl, X_scaler, y_scaler = create_ts_dl(time_series[["Close"]], time_series["Close"], lags=lags, horizon=horizon, stride=stride,\
                                            batch_size=batch_size, device=device, data_preprocess="log_returns",\
                                            val_size=val_size, test_size=test_size, drop_last=drop_last)

    synth_time_series = np.load(f"{synthetic_path}selected{ts_index}.npy")
    synth_dls = []
    for i in range(synth_time_series.shape[0]):
        synth_dl, *_ = create_ts_dl(synth_time_series[i].reshape(127, 1), synth_time_series[i], lags=lags, horizon=horizon, stride=stride,\
                                            batch_size=batch_size, device=device, data_preprocess=None,\
                                            val_size=0, test_size=0, drop_last=drop_last)
        synth_dls.append(synth_dl)
    
    model = Model(seed=0, device=device)
    model.set_model(TCN, **model_params)
    optim_params = {'params': model.model.parameters(), 'lr': 4e-4}
    model.set_optim(torch.optim.AdamW, **optim_params)
    model.set_criterion(MAE)

    # cdl = CombinedDataLoader(train_dl, *synth_dls)
    # only synth data
    cdl = CombinedDataLoader(*synth_dls)
    model.train(cdl, epochs=epochs, print_info=verbose, agg_loss="mean")
    results.append({"train": model.eval(train_dl, agg_loss="mean"), "val": model.eval(val_dl, agg_loss="mean")})

    del model, train_dl, val_dl, test_dl
    torch.cuda.empty_cache()

0it [00:00, ?it/s]Global seed set to 0
1it [00:33, 33.45s/it]Global seed set to 0
2it [01:02, 31.03s/it]Global seed set to 0
3it [01:31, 29.91s/it]Global seed set to 0
4it [01:59, 29.26s/it]Global seed set to 0
5it [02:26, 28.48s/it]Global seed set to 0
6it [02:54, 28.27s/it]Global seed set to 0
7it [03:22, 28.21s/it]Global seed set to 0
8it [03:51, 28.26s/it]Global seed set to 0
9it [04:20, 28.53s/it]Global seed set to 0
10it [04:49, 28.70s/it]Global seed set to 0
11it [05:13, 27.28s/it]Global seed set to 0
12it [05:42, 27.72s/it]Global seed set to 0
13it [06:10, 28.02s/it]Global seed set to 0
14it [06:39, 28.30s/it]Global seed set to 0
15it [07:08, 28.40s/it]Global seed set to 0
16it [07:36, 28.48s/it]Global seed set to 0
17it [08:06, 28.73s/it]Global seed set to 0
18it [08:35, 28.81s/it]Global seed set to 0
19it [09:04, 28.83s/it]Global seed set to 0
20it [09:37, 30.12s/it]Global seed set to 0
21it [10:05, 29.53s/it]Global seed set to 0
22it [10:35, 29.76s/it]Global seed set to 0
23

In [7]:
results

[{'train': 0.10775789991021156, 'val': 0.08761278539896011},
 {'train': 0.04027101770043373, 'val': 0.05313824117183685},
 {'train': 0.007714773993939161, 'val': 0.0056333988904953},
 {'train': 0.010017184230188528, 'val': 0.010736174881458282},
 {'train': 0.016770757269114256, 'val': 0.012860503979027271},
 {'train': 0.029067910586794216, 'val': 0.02728160098195076},
 {'train': 0.02904174029827118, 'val': 0.03444803133606911},
 {'train': 0.017426739819347857, 'val': 0.015151316300034523},
 {'train': 0.009081647576143345, 'val': 0.006351890275254846},
 {'train': 0.008119670208543539, 'val': 0.006900529842823744},
 {'train': 0.053747433341211744, 'val': 0.03965535759925842},
 {'train': 0.03879339247941971, 'val': 0.020823094062507153},
 {'train': 0.03564218493799368, 'val': 0.03145353216677904},
 {'train': 0.01201541442424059, 'val': 0.0064820037223398685},
 {'train': 0.018960770405828952, 'val': 0.010626457631587982},
 {'train': 0.04312667790800333, 'val': 0.02642847504466772},
 {'trai

In [8]:
pd.DataFrame(results).to_csv("results\\QuantGAN_synth_TCN.csv", index=False)

# Augmentation with FourierFlow synthetic data

In [4]:
synthetic_path = f"{dataset_path}synthetic/FourierFlow/"

In [7]:
ts_iterator = get_hsm_dataset(dataset_path, selected_files=f"{dataset_path}/selected.csv")
epochs = 2

results = []
for ts_index, time_series in tqdm(enumerate(ts_iterator)):
    train_dl, val_dl, test_dl, X_scaler, y_scaler = create_ts_dl(time_series[["Close"]], time_series["Close"], lags=lags, horizon=horizon, stride=stride,\
                                            batch_size=batch_size, device=device, data_preprocess="log_returns",\
                                            val_size=val_size, test_size=test_size, drop_last=drop_last)

    synth_time_series = np.load(f"{synthetic_path}selected{ts_index}.npy")
    synth_dls = []
    for i in range(synth_time_series.shape[0]):
        synth_dl, *_ = create_ts_dl(synth_time_series[i].reshape(- 1, 1), synth_time_series[i], lags=lags, horizon=horizon, stride=stride,\
                                            batch_size=batch_size, device=device, data_preprocess=None,\
                                            val_size=0, test_size=0, drop_last=drop_last)
        synth_dls.append(synth_dl)
    
    model = Model(seed=0, device=device)
    model.set_model(TCN, **model_params)
    optim_params = {'params': model.model.parameters(), 'lr': 4e-4}
    model.set_optim(torch.optim.AdamW, **optim_params)
    model.set_criterion(MAE)

    # only synth data
    cdl = CombinedDataLoader(*synth_dls)
    model.train(cdl, epochs=epochs, print_info=verbose, agg_loss="mean")
    results.append({"train": model.eval(train_dl, agg_loss="mean"), "val": model.eval(val_dl, agg_loss="mean")})

    del model, train_dl, val_dl, test_dl
    torch.cuda.empty_cache()

0it [00:00, ?it/s]Global seed set to 0
1it [00:34, 34.76s/it]Global seed set to 0
2it [00:55, 26.41s/it]Global seed set to 0
3it [01:16, 24.14s/it]Global seed set to 0
4it [01:39, 23.66s/it]Global seed set to 0
5it [02:00, 22.46s/it]Global seed set to 0
6it [02:21, 22.06s/it]Global seed set to 0
7it [02:43, 22.16s/it]Global seed set to 0
8it [03:06, 22.28s/it]Global seed set to 0
9it [03:27, 21.92s/it]Global seed set to 0
10it [03:47, 21.52s/it]Global seed set to 0
11it [04:07, 20.80s/it]Global seed set to 0
12it [04:26, 20.46s/it]Global seed set to 0
13it [04:47, 20.63s/it]Global seed set to 0
14it [05:07, 20.27s/it]Global seed set to 0
15it [05:27, 20.11s/it]Global seed set to 0
16it [05:46, 19.99s/it]Global seed set to 0
17it [06:06, 19.81s/it]Global seed set to 0
18it [06:27, 20.25s/it]Global seed set to 0
19it [06:46, 20.04s/it]Global seed set to 0
20it [07:06, 19.98s/it]Global seed set to 0
21it [07:27, 20.12s/it]Global seed set to 0
22it [07:47, 20.21s/it]Global seed set to 0
23

In [8]:
results

[{'train': 0.04299928620457649, 'val': 0.09553902596235275},
 {'train': 0.034737035632133484, 'val': 0.0577583909034729},
 {'train': 0.007313713897019625, 'val': 0.0054608541540801525},
 {'train': 0.010091571447749933, 'val': 0.01081530936062336},
 {'train': 0.0158067112788558, 'val': 0.0141794728115201},
 {'train': 0.027944813172022503, 'val': 0.02861202508211136},
 {'train': 0.027533545345067977, 'val': 0.037445973604917526},
 {'train': 0.016803331300616266, 'val': 0.015673236921429634},
 {'train': 0.009009471939255794, 'val': 0.006061585154384375},
 {'train': 0.008441495522856713, 'val': 0.007223269436508417},
 {'train': 0.0509502746992641, 'val': 0.04000464268028736},
 {'train': 0.035812276105086006, 'val': 0.020886201411485672},
 {'train': 0.03362286960085233, 'val': 0.03203243296593428},
 {'train': 0.011749354074709117, 'val': 0.006637847051024437},
 {'train': 0.01852165637537837, 'val': 0.010446077212691307},
 {'train': 0.04027140736579895, 'val': 0.027284150943160057},
 {'train

In [9]:
pd.DataFrame(results).to_csv("results\\FourierFlow_synth_TCN.csv", index=False)

# Augmentation with RealNVP synthetic data

In [16]:
synthetic_path = f"{dataset_path}synthetic/RealNVP/"
ts_iterator = get_hsm_dataset(dataset_path, selected_files=f"{dataset_path}/selected.csv")
epochs = 2

results = []
for ts_index, time_series in tqdm(enumerate(ts_iterator)):
    train_dl, val_dl, test_dl, X_scaler, y_scaler = create_ts_dl(time_series[["Close"]], time_series["Close"], lags=lags, horizon=horizon, stride=stride,\
                                            batch_size=batch_size, device=device, data_preprocess="log_returns",\
                                            val_size=val_size, test_size=test_size, drop_last=drop_last)

    synth_time_series = np.load(f"{synthetic_path}selected{ts_index}.npy")
    synth_dls = []
    for i in range(synth_time_series.shape[0]):
        synth_dl, *_ = create_ts_dl(synth_time_series[i].reshape(- 1, 1), synth_time_series[i], lags=lags, horizon=horizon, stride=stride,\
                                            batch_size=batch_size, device=device, data_preprocess=None,\
                                            val_size=0, test_size=0, drop_last=drop_last)
        synth_dls.append(synth_dl)
    
    model = Model(seed=0, device=device)
    model.set_model(TCN, **model_params)
    optim_params = {'params': model.model.parameters(), 'lr': 4e-4}
    model.set_optim(torch.optim.AdamW, **optim_params)
    model.set_criterion(MAE)

    # only synth data
    cdl = CombinedDataLoader(*synth_dls)
    model.train(cdl, epochs=epochs, print_info=verbose, agg_loss="mean")
    results.append({"train": model.eval(train_dl, agg_loss="mean"), "val": model.eval(val_dl, agg_loss="mean")})

    del model, train_dl, val_dl, test_dl
    torch.cuda.empty_cache()

0it [00:00, ?it/s]Global seed set to 0
1it [00:25, 25.05s/it]Global seed set to 0
2it [00:44, 21.77s/it]Global seed set to 0
3it [01:04, 20.89s/it]Global seed set to 0
4it [01:26, 21.49s/it]Global seed set to 0
5it [01:48, 21.63s/it]Global seed set to 0
6it [02:12, 22.22s/it]Global seed set to 0
7it [02:36, 22.82s/it]Global seed set to 0
8it [02:57, 22.49s/it]Global seed set to 0
9it [03:19, 22.25s/it]Global seed set to 0
10it [03:40, 21.81s/it]Global seed set to 0
11it [04:01, 21.66s/it]Global seed set to 0
12it [04:22, 21.29s/it]Global seed set to 0
13it [04:44, 21.48s/it]Global seed set to 0
14it [05:04, 21.27s/it]Global seed set to 0
15it [05:23, 20.54s/it]Global seed set to 0
16it [05:40, 19.39s/it]Global seed set to 0
17it [05:57, 18.60s/it]Global seed set to 0
18it [06:15, 18.39s/it]Global seed set to 0
19it [06:34, 18.60s/it]Global seed set to 0
20it [06:53, 18.94s/it]Global seed set to 0
21it [07:10, 18.35s/it]Global seed set to 0
22it [07:29, 18.55s/it]Global seed set to 0
23

In [17]:
results

[{'train': 0.022699295543134212, 'val': 0.11058970540761948},
 {'train': 0.031208307792743046, 'val': 0.05794366076588631},
 {'train': 0.0069389975008865195, 'val': 0.0051651690155267715},
 {'train': 0.009290863604595264, 'val': 0.010841844603419304},
 {'train': 0.014406627044081688, 'val': 0.013607817701995373},
 {'train': 0.02649128871659438, 'val': 0.027409205213189125},
 {'train': 0.025554822757840157, 'val': 0.03524057939648628},
 {'train': 0.016241544485092164, 'val': 0.015404561534523964},
 {'train': 0.00858282313371698, 'val': 0.006164876976981759},
 {'train': 0.007948750723153352, 'val': 0.007010172121226788},
 {'train': 0.04852983603874842, 'val': 0.03915001079440117},
 {'train': 0.034994653736551605, 'val': 0.021006189286708832},
 {'train': 0.03291084306935469, 'val': 0.031834215857088566},
 {'train': 0.011516043683513999, 'val': 0.00660939491353929},
 {'train': 0.017801154404878616, 'val': 0.010262904688715935},
 {'train': 0.03930122163146734, 'val': 0.027040889486670494},


In [18]:
pd.DataFrame(results).to_csv("results\\RealNVP_synth_TCN.csv", index=False)